# Figure to show the new 2-halo term and miscentering functionality

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [ ]:
from paper_formating import add_grid, prep_plot

Imports specific to clmm 

In [ ]:
import os

os.environ["CLMM_MODELING_BACKEND"] = (
    "ccl"  # here you may choose ccl, nc (NumCosmo) or ct (cluster_toolkit)
)

import clmm
import clmm.utils as u
from clmm import Cosmology

Make sure we know which version we're using

In [ ]:
clmm.__version__

Define a cosmology using astropy

In [ ]:
cosmo = Cosmology(H0=67.0, Omega_dm0=0.315 - 0.045, Omega_b0=0.045, Omega_k0=0.0)

Define the galaxy cluster model.  Here, we choose parameters that describe the galaxy cluster model, including the mass definition, concentration, and mass distribution.  For the mass distribution, we choose a distribution that follows an NFW profile.

In [ ]:
moo = clmm.Modeling(massdef="mean", delta_mdef=200, halo_profile_model="nfw")

mass_cl = 1.0e14
z_cl = 0.4

conc_cl = 5.4  # Duffy08 value for this halo mass and redshift (see last commented cell)
halo_bias = (
    2.4  # Tkinker10 value for this halo mass and redshift (see last commented cell)
)

moo.set_cosmo(cosmo)
moo.set_concentration(conc_cl)
moo.set_mass(mass_cl)

In [ ]:
r_proj = np.logspace(-2, 2, 100)

DeltaSigma = moo.eval_excess_surface_density(r_proj, z_cl)

# Miscentered DeltaSigma
DeltaSigma_mis = moo.eval_excess_surface_density(r_proj, z_cl, r_mis=0.2)

# 2halo DeltaSigma
DeltaSigma_2h = moo.eval_excess_surface_density_2h(r_proj, z_cl, halobias=6)

# boost model
r_scale = 0.3
nfw_boost = u.compute_nfw_boost(r_proj, r_scale, boost0=0.2)

Plot the predicted profiles

In [ ]:
fig = prep_plot(figsize=(9, 9))
ax = plt.axes()

ax.loglog(r_proj, DeltaSigma, label="1-halo (reference)", color="k")
ax.loglog(r_proj, DeltaSigma_mis, ls="--", label=r"1-halo ($R_{\rm mis} = 0.2$Mpc)")
ax.loglog(r_proj, DeltaSigma / nfw_boost, ls=":", label="1-halo (boost correction)")
ax.loglog(r_proj, DeltaSigma_2h, ls="dashdot", label="2-halo")
ax.legend(loc=1, fontsize=6.5)

ax.set_xlabel("R [Mpc]")
ax.set_ylabel(r"$\Delta\Sigma$ [M$_\odot$ Mpc$^{-2}$]")
ax.set_ylim(8.0e10, 1e15)
ax.set_xlim(1e-2, 1e2)

add_grid(ax)

fig.tight_layout()
fig.savefig("2h_miscentering_boost.png")